In [1]:
from utils import generate_image, create_character_dataloader, load_images
from chatgpt import chat, signification, gaze, grasp
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from gensim.models import Word2Vec
import random

At the first, the agent needs to learn about the identities and images of 10 characters around it.

In [2]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def trainmlp(input_size, output_size, lr,epochs):
    mlp = MLP(input_size,output_size)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(mlp.parameters(), lr=lr)
    dataloader, id2vec, cnn = create_character_dataloader(batch_size=4, shuffle=True)
    for epoch in range(epochs):
         for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = mlp(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
    return mlp, id2vec, cnn

In [3]:
mlp, id2vec, cnn = trainmlp(4, 4, 0.001, 1000)

c:\Users\lilin\Desktop\FreeAgent\FreeAgent\utils.py:118: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  self.input_vectors = torch.tensor(input_vectors, dtype=torch.float32)


Epoch 1/1000, Loss: 197.4747314453125
Epoch 1/1000, Loss: 99.67565155029297
Epoch 1/1000, Loss: 104.63801574707031
Epoch 2/1000, Loss: 62.86069869995117
Epoch 2/1000, Loss: 33.93539810180664
Epoch 2/1000, Loss: 10.639435768127441
Epoch 3/1000, Loss: 13.28282642364502
Epoch 3/1000, Loss: 9.96271800994873
Epoch 3/1000, Loss: 6.592103004455566
Epoch 4/1000, Loss: 10.190598487854004
Epoch 4/1000, Loss: 8.44270133972168
Epoch 4/1000, Loss: 14.907903671264648
Epoch 5/1000, Loss: 14.176919937133789
Epoch 5/1000, Loss: 16.07428741455078
Epoch 5/1000, Loss: 16.79055404663086
Epoch 6/1000, Loss: 20.166500091552734
Epoch 6/1000, Loss: 12.964554786682129
Epoch 6/1000, Loss: 10.51151180267334
Epoch 7/1000, Loss: 12.882051467895508
Epoch 7/1000, Loss: 10.676977157592773
Epoch 7/1000, Loss: 8.195237159729004
Epoch 8/1000, Loss: 8.131013870239258
Epoch 8/1000, Loss: 6.707973003387451
Epoch 8/1000, Loss: 4.74715518951416
Epoch 9/1000, Loss: 5.025655269622803
Epoch 9/1000, Loss: 4.179853439331055
Epoch 

Then the agent identities with one of them randomly to limit the complexity. A more delicate designation should model this selection process in the context of planning with expected free energy. In essence, the identification begin with the imaginary identification whose logic is mirror and unity.

In [5]:
def get_identify_object():
    with open('10characters.json') as file:
        characterdata = json.load(file)
        names = [character['name'] for character in characterdata['characters']]
    imaginary_id = random.choice(names)
    image = load_images([f'character_images/{imaginary_id}.png'])
    image_id = torch.tensor(np.array(cnn(image))[0])
    symbo_vec = mlp(image_id).detach().numpy()
    symbo_id = id2vec.wv.similar_by_vector(symbo_vec)[0][0]
    return imaginary_id, image_id, symbo_id


def metonymy(timesteps):
    imgi, ima, symb = get_identify_object()
    intention = {"intention": "I want to be like " + imgi + " who is a " + symb + "."}
    outputs = []
    symb_id = symb

    for i in range(timesteps):
        IO = signification(symb_id)
        io = gaze(IO)
        symb_id = grasp(symb,io)

        outputs.append({
            "I(O)": IO,
            "i(o)": io,
            "I'(O)": symb_id
        })

    data_to_store = {
        "intention": intention,
        "timesteps_outputs": outputs
    }

    with open('output.json', 'w') as json_file:
        json.dump(data_to_store, json_file, indent=4)


In [7]:
metonymy(10)